In [1]:
import numpy as np
import pandas as pd
import time

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [2]:
import numpy as np
import pandas as pd

class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None):  
        """
        Initializes Linear Regression class.
        ---
        Parameters: 
            alpha: Learning Rate, defaults to 1e-10.
            num_iter: Number of Iterations to update coefficient with training data.
            early_stop: Constant control early_stop.
            intercept: Bool, If we are going to fit a intercept, default True.
            init_weight: Matrix (n x 1), input init_weight for testing.
        """
        self.model_name = 'Linear Regression'
        self.alpha = alpha
        self.num_iter = num_iter
        self.early_stop = early_stop
        self.intercept = intercept
        self.init_weight = init_weight  ### For testing correctness.
        self.coef = None
        self.loss = []
        

    def __str__(self):
        return self.model_name
    

    def fit(self, X_train, y_train):
        """
        Saves datasets to our model, and performs gradient descent.
        ---
        Parameters:
            X_train: Matrix or 2-D array. Input feature matrix.
            y_train: Matrix or 2-D array. Input target value.
        """
        self.X = np.mat(X_train)
        if isinstance(y_train, pd.core.series.Series):
            self.y = np.reshape(y_train.values, (-1, 1))
        elif y_train.ndim == 1:
            self.y = np.reshape(y_train, (-1, 1))
        else:
            self.y = y_train
        
        # adds column of all 1's to first column of X if intercept is True.
        if self.intercept:
            ones = np.ones((self.X.shape[0], 1))
            self.X = np.hstack((ones, self.X))
        
        if self.init_weight is not None:
            self.coef = self.init_weight
        else:
            # initializes coefficient with uniform from [-1, 1]
            self.coef = np.random.rand(self.X.shape[1]) * 2 - 1
            self.coef = np.reshape(self.coef, (-1, 1))

        # Call gradient_descent function to train.
        self.gradient_descent()
        

    def gradient(self):
        """
        Helper function to calculate the gradient of the cost function.
        Gradient: -2X^T(y - Xw)
        ---
        Returns the gradient of the cost function.
        """
        y_pred = np.dot(self.X, self.coef)
        y_pred = np.reshape(y_pred, (-1, 1))
        error = np.subtract(self.y, y_pred)

        # Return gradient of cost function
        return -2 * np.dot(self.X.T, error)
        

    def gradient_descent(self):
        """
        Performs gradient descent to find the model's optimal coefficients.
        Cost function: SE = \sum^{n}_{i=1}(Y_i - \hat{Y}_i)^2
        """
        for i in range(self.num_iter):
            if i == 0:
                pre_error = np.sum(np.power(np.dot(self.X, self.coef) - self.y, 2))
                
            # Calculate the gradient and temporary coefficient to compare
            temp_coef = self.coef - self.alpha * self.gradient()  

            # Calculate error for early stopping
            y_pred = np.dot(self.X, temp_coef)
            current_error = np.sum(np.power(y_pred - self.y, 2))

            ### This is the early stop, don't modify the following three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                self.coef = temp_coef
                return self
            
            # Update learning rate and coefficients based on error comparison
            if current_error < pre_error:
                self.coef = temp_coef
                # Increase learning rate if error decreases
                self.alpha *= 1.3 
            else:
                # Decrease learning rate if error increases
                self.alpha *= 0.9 

            # Update previous error for next iteration
            pre_error = min(pre_error, current_error)
            # Add loss to loss list we create
            self.loss.append(min(pre_error, current_error))

            if i % 1000 == 0:
                print('Iteration: ' + str(i))
                print('Coef:      ' + str(self.coef))
                print('Loss:      ' + str(current_error))
                print('Alpha:     ' + str(self.alpha)) 
    
        return self


    def ind_predict(self, x: list):
        """
        Predict the value based on its feature vector x.
        ---
        Parameters:
            x: Matrix, array or list. Input feature point.
        ---
        Returns a prediction of given data point.
        """
        x = np.matrix(x)
        return np.array(np.dot(x, self.coef)).flatten()


    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            ---
            Parameters:
                X: Matrix, array or list. Input feature point.
            ---
            Returns a prediction of the given data matrix.
        """
        X = np.matrix(X)

        # Adds ones if intercept is True.
        if self.intercept:
            ones = np.ones((X.shape[0], 1))
            X = np.hstack((ones, X))
            
        # Calls ind_predict for each value in the X matrix.
        predictions = np.array([self.ind_predict(x) for x in X]).flatten()
        return predictions



In [3]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [4]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() + 20

#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [5]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

Iteration: 0
Coef:      [[15]
 [25]]
Loss:      2.931949540902693e+25
Alpha:     0.9
Iteration: 1000
Coef:      [[15.00927684]
 [30.00748189]]
Loss:      1250.9887008778671
Alpha:     1.60528649192378e-08
Iteration: 2000
Coef:      [[15.0114471 ]
 [30.00751175]]
Loss:      1249.8963903412282
Alpha:     1.9151548451188908e-08
Iteration: 3000
Coef:      [[15.01360825]
 [30.0075084 ]]
Loss:      1248.824872042921
Alpha:     1.581810277533585e-08
Iteration: 4000
Coef:      [[15.0157756]
 [30.0074717]]
Loss:      1247.7244230806502
Alpha:     1.8871470185031235e-08
Iteration: 5000
Coef:      [[15.01793566]
 [30.00746476]]
Loss:      1246.6650689874657
Alpha:     1.5586773866839906e-08
Iteration: 6000
Coef:      [[15.02010111]
 [30.00749851]]
Loss:      1245.5988198093428
Alpha:     1.8595487871499764e-08
Iteration: 7000
Coef:      [[15.02225851]
 [30.00749529]]
Loss:      1244.4920314864953
Alpha:     1.5358827985036065e-08
Iteration: 8000
Coef:      [[15.02442272]
 [30.00745634]]
Loss:    

Iteration: 71000
Coef:      [[15.15865651]
 [30.00729126]]
Loss:      1177.2200912681312
Alpha:     1.9999479714469813e-08
Iteration: 72000
Coef:      [[15.1607536]
 [30.0072864]]
Loss:      1176.2115816806256
Alpha:     1.6518446348242352e-08
Iteration: 73000
Coef:      [[15.16285697]
 [30.00725145]]
Loss:      1175.1713204803884
Alpha:     1.9707001034912428e-08
Iteration: 74000
Coef:      [[15.16495375]
 [30.00724227]]
Loss:      1174.1917864000516
Alpha:     1.6276875395135137e-08
Iteration: 75000
Coef:      [[15.16705455]
 [30.00727556]]
Loss:      1173.1326985783758
Alpha:     1.9418799655525682e-08
Iteration: 76000
Coef:      [[15.16914797]
 [30.00727123]]
Loss:      1172.1214534470412
Alpha:     1.603883725160054e-08
Iteration: 77000
Coef:      [[15.17124908]
 [30.00723573]]
Loss:      1171.1041575817992
Alpha:     1.913481302372701e-08
Iteration: 78000
Coef:      [[15.17334073]
 [30.00723417]]
Loss:      1170.0963468418197
Alpha:     1.5804280252720698e-08
Iteration: 79000
Coe

Iteration: 141000
Coef:      [[15.3035568 ]
 [30.00707177]]
Loss:      1107.824626671242
Alpha:     1.724979750322579e-08
Iteration: 142000
Coef:      [[15.30559149]
 [30.00706897]]
Loss:      1106.8452321954228
Alpha:     1.4247363363604124e-08
Iteration: 143000
Coef:      [[15.30763202]
 [30.0070336 ]]
Loss:      1105.8964431862082
Alpha:     1.6997531040877536e-08
Iteration: 144000
Coef:      [[15.30967236]
 [30.00706296]]
Loss:      1104.921578797435
Alpha:     2.0278563416417953e-08
Iteration: 145000
Coef:      [[15.31170371]
 [30.00705732]]
Loss:      1103.9709411640222
Alpha:     1.6748953802591952e-08
Iteration: 146000
Coef:      [[15.31374279]
 [30.00702119]]
Loss:      1103.0091723669068
Alpha:     1.9982003329054106e-08
Iteration: 147000
Coef:      [[15.31577245]
 [30.00702125]]
Loss:      1102.0600520184048
Alpha:     1.650401183599643e-08
Iteration: 148000
Coef:      [[15.31780824]
 [30.00704716]]
Loss:      1101.0845854507731
Alpha:     1.9689780229651923e-08
Iteration: 1

Iteration: 210000
Coef:      [[15.44214603]
 [30.0068641 ]]
Loss:      1043.4188864631974
Alpha:     1.8013529095244246e-08
Iteration: 211000
Coef:      [[15.4441204 ]
 [30.00685994]]
Loss:      1042.4868199012012
Alpha:     1.4878162739754173e-08
Iteration: 212000
Coef:      [[15.4461007 ]
 [30.00682634]]
Loss:      1041.5944293786383
Alpha:     1.7750093581962635e-08
Iteration: 213000
Coef:      [[15.44807408]
 [30.00682115]]
Loss:      1040.6805978798589
Alpha:     1.4660579809873497e-08
Iteration: 214000
Coef:      [[15.4500527 ]
 [30.00685055]]
Loss:      1039.788466358443
Alpha:     1.7490510632456384e-08
Iteration: 215000
Coef:      [[15.45202361]
 [30.00684675]]
Loss:      1038.8674348102413
Alpha:     1.4446178881171554e-08
Iteration: 216000
Coef:      [[15.4540018 ]
 [30.00681004]]
Loss:      1038.0037384670206
Alpha:     1.7234723905621428e-08
Iteration: 217000
Coef:      [[15.45597757]
 [30.00684248]]
Loss:      1037.0647429669702
Alpha:     2.056154160531273e-08
Iteration:

Iteration: 282000
Coef:      [[15.58239813]
 [30.00662062]]
Loss:      980.1688460729881
Alpha:     1.5309674267708156e-08
Iteration: 283000
Coef:      [[15.58431908]
 [30.00665106]]
Loss:      979.3485409033551
Alpha:     1.8264899753722897e-08
Iteration: 284000
Coef:      [[15.58623156]
 [30.00664585]]
Loss:      978.4655267729675
Alpha:     1.508578077756729e-08
Iteration: 285000
Coef:      [[15.58815063]
 [30.00661119]]
Loss:      977.6409658698716
Alpha:     1.7997788116895945e-08
Iteration: 286000
Coef:      [[15.59006219]
 [30.00660729]]
Loss:      976.7740527195891
Alpha:     1.4865161576222587e-08
Iteration: 287000
Coef:      [[15.59197891]
 [30.00663704]]
Loss:      975.9346340780656
Alpha:     1.7734582804631006e-08
Iteration: 288000
Coef:      [[15.59388951]
 [30.00663905]]
Loss:      975.0857746560125
Alpha:     1.4647768779445198e-08
Iteration: 289000
Coef:      [[15.59580436]
 [30.00659964]]
Loss:      974.2478764811975
Alpha:     1.7475226689609295e-08
Iteration: 290000

...

Iteration: 9710000
Coef:      [[19.92625615]
 [30.00011107]]
Loss:      0.27313909183255336
Alpha:     1.61252771273656e-08
Iteration: 9711000
Coef:      [[19.92628821]
 [30.00011046]]
Loss:      0.2728975976589415
Alpha:     1.9237938383415638e-08
Iteration: 9712000
Coef:      [[19.92632013]
 [30.00011045]]
Loss:      0.27266178206302705
Alpha:     1.588945600456426e-08
Iteration: 9713000
Coef:      [[19.92635217]
 [30.00011092]]
Loss:      0.2724241920945766
Alpha:     1.8956596723726476e-08
Iteration: 9714000
Coef:      [[19.92638408]
 [30.00011088]]
Loss:      0.27219160723451236
Alpha:     1.565708360400928e-08
Iteration: 9715000
Coef:      [[19.92641607]
 [30.00011032]]
Loss:      0.27194976238460766
Alpha:     1.867936949292768e-08
Iteration: 9716000
Coef:      [[19.92644796]
 [30.00011021]]
Loss:      0.27171852479473535
Alpha:     1.5428109490502333e-08
Iteration: 9717000
Coef:      [[19.92647993]
 [30.0001107 ]]
Loss:      0.2714862003933215
Alpha:     1.8406196520317572e-08


Iteration: 9782000
Coef:      [[19.92852547]
 [30.00010762]]
Loss:      0.2565825201618144
Alpha:     1.9795896535353553e-08
Iteration: 9783000
Coef:      [[19.92855644]
 [30.00010762]]
Loss:      0.25636629639681513
Alpha:     1.6350298082905063e-08
Iteration: 9784000
Coef:      [[19.92858748]
 [30.00010706]]
Loss:      0.25613667978197596
Alpha:     1.9506395120217923e-08
Iteration: 9785000
Coef:      [[19.92861843]
 [30.00010697]]
Loss:      0.25591915963999157
Alpha:     1.611118618289905e-08
Iteration: 9786000
Coef:      [[19.92864947]
 [30.00010748]]
Loss:      0.25569471041987046
Alpha:     1.922112746480189e-08
Iteration: 9787000
Coef:      [[19.92868037]
 [30.00010737]]
Loss:      0.2554730322256715
Alpha:     1.5875571130499985e-08
Iteration: 9788000
Coef:      [[19.92871138]
 [30.00010684]]
Loss:      0.2552496955280562
Alpha:     1.8940031653272208e-08
Iteration: 9789000
Coef:      [[19.92874226]
 [30.00010681]]
Loss:      0.2550300670118777
Alpha:     1.564340178670906e-08

Iteration: 9854000
Coef:      [[19.93072484]
 [30.00010386]]
Loss:      0.24103608520397948
Alpha:     1.682450596943404e-08
Iteration: 9855000
Coef:      [[19.93075498]
 [30.00010431]]
Loss:      0.24082657783151024
Alpha:     2.007213932603329e-08
Iteration: 9856000
Coef:      [[19.93078497]
 [30.00010423]]
Loss:      0.24062031002606518
Alpha:     1.6578459104195533e-08
Iteration: 9857000
Coef:      [[19.93081505]
 [30.00010371]]
Loss:      0.24040676756009208
Alpha:     1.9778598049470846e-08
Iteration: 9858000
Coef:      [[19.93084504]
 [30.00010359]]
Loss:      0.2402059441713649
Alpha:     1.6336010505676093e-08
Iteration: 9859000
Coef:      [[19.93087509]
 [30.00010405]]
Loss:      0.23998939009549028
Alpha:     1.948934961283169e-08
Iteration: 9860000
Coef:      [[19.93090504]
 [30.00010403]]
Loss:      0.23978426118914226
Alpha:     1.6097107551691725e-08
Iteration: 9861000
Coef:      [[19.93093508]
 [30.00010352]]
Loss:      0.23957378534628077
Alpha:     1.9204331236275093e

Iteration: 9926000
Coef:      [[19.93285655]
 [30.0001011 ]]
Loss:      0.22642898371784334
Alpha:     1.4299125104538296e-08
Iteration: 9927000
Coef:      [[19.93288575]
 [30.00010056]]
Loss:      0.2262380013950245
Alpha:     1.7059284347493237e-08
Iteration: 9928000
Coef:      [[19.93291482]
 [30.00010052]]
Loss:      0.22603688349961767
Alpha:     1.4090010203095355e-08
Iteration: 9929000
Coef:      [[19.93294398]
 [30.00010096]]
Loss:      0.22584259989434424
Alpha:     1.6809804009435302e-08
Iteration: 9930000
Coef:      [[19.93297315]
 [30.0001004 ]]
Loss:      0.22564523814011458
Alpha:     2.005459944759667e-08
Iteration: 9931000
Coef:      [[19.93300217]
 [30.00010041]]
Loss:      0.225451283029536
Alpha:     1.6563972150282445e-08
Iteration: 9932000
Coef:      [[19.93303129]
 [30.0001008 ]]
Loss:      0.2252508920307095
Alpha:     1.9761314679731322e-08
Iteration: 9933000
Coef:      [[19.93306032]
 [30.00010086]]
Loss:      0.22506430438008873
Alpha:     1.6321735413531974e-

Iteration: 9998000
Coef:      [[19.93492276]
 [30.00009798]]
Loss:      0.21271129825693205
Alpha:     1.755405497094644e-08
Iteration: 9999000
Coef:      [[19.93495096]
 [30.00009797]]
Loss:      0.2125244361730363
Alpha:     1.4498662933810355e-08


####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [6]:
clf.coef

matrix([[19.93497924],
        [30.00009741]])

In [7]:
np.array(clf.predict(X))

array([   49.93507666,   199.93556373,   349.9360508 ,   499.93653787,
         649.93702495,   799.93751202,   949.93799909,  1099.93848616,
        1249.93897323,  1399.93946031,  1549.93994738,  1699.94043445,
        1849.94092152,  1999.9414086 ,  2149.94189567,  2299.94238274,
        2449.94286981,  2599.94335688,  2749.94384396,  2899.94433103,
        3049.9448181 ,  3199.94530517,  3349.94579225,  3499.94627932,
        3649.94676639,  3799.94725346,  3949.94774053,  4099.94822761,
        4249.94871468,  4399.94920175,  4549.94968882,  4699.9501759 ,
        4849.95066297,  4999.95115004,  5149.95163711,  5299.95212418,
        5449.95261126,  5599.95309833,  5749.9535854 ,  5899.95407247,
        6049.95455955,  6199.95504662,  6349.95553369,  6499.95602076,
        6649.95650784,  6799.95699491,  6949.95748198,  7099.95796905,
        7249.95845612,  7399.9589432 ,  7549.95943027,  7699.95991734,
        7849.96040441,  7999.96089149,  8149.96137856,  8299.96186563,
      

#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
y = wine.quality

In [10]:
lr = LinearRegression()
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

800.6676988774321

In [13]:
lr.coef_

array([34.82170159,  0.39144139])

In [11]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

Iteration: 0
Coef:      [[-0.81031599]
 [-0.18615441]
 [-0.02846642]]
Loss:      9822201510479968.0
Alpha:     0.9
Iteration: 1000
Coef:      [[-0.62869919]
 [-0.00359366]
 [ 0.5972751 ]]
Loss:      910.5040653553699
Alpha:     5.764558317550444e-06
Iteration: 2000
Coef:      [[-0.50420384]
 [ 0.12213461]
 [ 0.57650715]]
Loss:      890.7939792980912
Alpha:     6.87729065644642e-06
Iteration: 3000
Coef:      [[-0.39275379]
 [ 0.23473789]
 [ 0.55510766]]
Loss:      874.3414214562089
Alpha:     5.680255604228824e-06
Iteration: 4000
Coef:      [[-0.29223654]
 [ 0.33634434]
 [ 0.53355982]]
Loss:      861.7966015338934
Alpha:     6.776714995536758e-06
Iteration: 5000
Coef:      [[-0.20191063]
 [ 0.42769177]
 [ 0.51669457]]
Loss:      850.7381545764861
Alpha:     5.5971857602931585e-06
Iteration: 6000
Coef:      [[-0.12010625]
 [ 0.51046537]
 [ 0.50299653]]
Loss:      842.4636556826954
Alpha:     6.677610184715118e-06
Iteration: 7000
Coef:      [[-0.04692764]
 [ 0.58455971]
 [ 0.48890356]]
Lo

Iteration: 65000
Coef:      [[0.60323458]
 [1.26557106]
 [0.3618264 ]]
Loss:      805.4991784080771
Alpha:     7.506312802448292e-06
Iteration: 66000
Coef:      [[0.60315466]
 [1.2659519 ]
 [0.36179866]]
Loss:      805.499083824599
Alpha:     6.199792548135955e-06
Iteration: 67000
Coef:      [[0.60305986]
 [1.26631955]
 [0.36177869]]
Loss:      805.49899263977
Alpha:     7.396538123899113e-06
Iteration: 68000
Coef:      [[0.60295135]
 [1.26667196]
 [0.36175628]]
Loss:      805.4989101665473
Alpha:     6.109124832580428e-06
Iteration: 69000
Coef:      [[0.60282969]
 [1.26701239]
 [0.36172924]]
Loss:      805.4988256192231
Alpha:     7.288368824764271e-06
Iteration: 70000
Coef:      [[0.60269794]
 [1.26734141]
 [0.36170998]]
Loss:      805.4987493270091
Alpha:     6.019783070204819e-06
Iteration: 71000
Coef:      [[0.60255657]
 [1.26766197]
 [0.36169916]]
Loss:      805.4986704196845
Alpha:     7.1817814274704295e-06
Iteration: 72000
Coef:      [[0.60240618]
 [1.26797206]
 [0.36168409]]


Iteration: 130000
Coef:      [[0.58973071]
 [1.28202289]
 [0.36155774]]
Loss:      805.4947258161825
Alpha:     5.589036374941192e-06
Iteration: 131000
Coef:      [[0.58949895]
 [1.28225263]
 [0.36155284]]
Loss:      805.4946615256938
Alpha:     6.667887724007881e-06
Iteration: 132000
Coef:      [[0.58926845]
 [1.282482  ]
 [0.36155276]]
Loss:      805.4945942583443
Alpha:     5.507300549695683e-06
Iteration: 133000
Coef:      [[0.58903777]
 [1.2827126 ]
 [0.36155845]]
Loss:      805.4945289735147
Alpha:     6.570374437422451e-06
Iteration: 134000
Coef:      [[0.58880729]
 [1.28294199]
 [0.36155888]]
Loss:      805.4944618354061
Alpha:     5.4267600548578805e-06
Iteration: 135000
Coef:      [[0.58857557]
 [1.28317155]
 [0.36155384]]
Loss:      805.494396239029
Alpha:     6.474287215800052e-06
Iteration: 136000
Coef:      [[0.58834474]
 [1.2834021 ]
 [0.36155886]]
Loss:      805.4943288415093
Alpha:     7.724018480446676e-06
Iteration: 137000
Coef:      [[0.58811423]
 [1.28363146]
 [0.3

Iteration: 195000
Coef:      [[0.57471989]
 [1.29694706]
 [0.3615712 ]]
Loss:      805.4904328253596
Alpha:     6.0110183920118995e-06
Iteration: 196000
Coef:      [[0.57448808]
 [1.29717642]
 [0.3615658 ]]
Loss:      805.4903654037772
Alpha:     7.1713249039828645e-06
Iteration: 197000
Coef:      [[0.57425772]
 [1.29740548]
 [0.3615665 ]]
Loss:      805.4902995494916
Alpha:     5.9231113690696695e-06
Iteration: 198000
Coef:      [[0.57402681]
 [1.29763604]
 [0.36157209]]
Loss:      805.4902337952672
Alpha:     7.066449193787344e-06
Iteration: 199000
Coef:      [[0.57379635]
 [1.297865  ]
 [0.36157171]]
Loss:      805.4901679176537
Alpha:     5.836489926070577e-06
Iteration: 200000
Coef:      [[0.57356464]
 [1.29809439]
 [0.36156701]]
Loss:      805.4901027129274
Alpha:     6.963107218952619e-06
Iteration: 201000
Coef:      [[0.57333402]
 [1.29832352]
 [0.36156665]]
Loss:      805.490036278816
Alpha:     5.7511352622893025e-06
Iteration: 202000
Coef:      [[0.5731033 ]
 [1.29855388]
 [

Iteration: 260000
Coef:      [[0.55971417]
 [1.31186267]
 [0.36158408]]
Loss:      805.4861421684487
Alpha:     6.464860789081134e-06
Iteration: 261000
Coef:      [[0.55948251]
 [1.312092  ]
 [0.3615793 ]]
Loss:      805.4860748036771
Alpha:     7.712772471155675e-06
Iteration: 262000
Coef:      [[0.55925204]
 [1.31232102]
 [0.36157915]]
Loss:      805.4860110029315
Alpha:     6.37031663222756e-06
Iteration: 263000
Coef:      [[0.5590214 ]
 [1.31255135]
 [0.36158504]]
Loss:      805.4859431475763
Alpha:     7.599978461496508e-06
Iteration: 264000
Coef:      [[0.55879108]
 [1.31278016]
 [0.36158455]]
Loss:      805.4858774164828
Alpha:     6.277155118850271e-06
Iteration: 265000
Coef:      [[0.55855925]
 [1.31300961]
 [0.36157952]]
Loss:      805.4858115160187
Alpha:     7.4888339868991556e-06
Iteration: 266000
Coef:      [[0.55832906]
 [1.31323857]
 [0.36158052]]
Loss:      805.4857457278747
Alpha:     6.1853560287363844e-06
Iteration: 267000
Coef:      [[0.5580983 ]
 [1.31346889]
 [0.

...

Iteration: 3985000
Coef:      [[-0.2891624 ]
 [ 2.15564552]
 [ 0.36233111]]
Loss:      805.2465384661689
Alpha:     6.190164649906525e-06
Iteration: 3986000
Coef:      [[-0.28938734]
 [ 2.15587009]
 [ 0.36233651]]
Loss:      805.2464751824174
Alpha:     7.385051753064256e-06
Iteration: 3987000
Coef:      [[-0.28961191]
 [ 2.15609329]
 [ 0.36233655]]
Loss:      805.2464135067228
Alpha:     6.099637735761882e-06
Iteration: 3988000
Coef:      [[-0.28983774]
 [ 2.15631679]
 [ 0.36233162]]
Loss:      805.2463494705971
Alpha:     7.27705043422802e-06
Iteration: 3989000
Coef:      [[-0.29006242]
 [ 2.15654004]
 [ 0.36233138]]
Loss:      805.2462884417762
Alpha:     6.010434715673056e-06
Iteration: 3990000
Coef:      [[-0.29028722]
 [ 2.15676453]
 [ 0.36233707]]
Loss:      805.2462242041123
Alpha:     7.17062856063609e-06
Iteration: 3991000
Coef:      [[-0.29051179]
 [ 2.15698773]
 [ 0.36233716]]
Loss:      805.2461622008841
Alpha:     5.922536228597125e-06
Iteration: 3992000
Coef:      [[-0.2

Iteration: 4045000
Coef:      [[-0.30265857]
 [ 2.16906169]
 [ 0.36234819]]
Loss:      805.2427783618438
Alpha:     5.747226269073054e-06
Iteration: 4046000
Coef:      [[-0.3028843 ]
 [ 2.16928508]
 [ 0.36234314]]
Loss:      805.242716664774
Alpha:     6.856613003713211e-06
Iteration: 4047000
Coef:      [[-0.30310869]
 [ 2.16950819]
 [ 0.36234372]]
Loss:      805.242652111036
Alpha:     5.663177025077899e-06
Iteration: 4048000
Coef:      [[-0.30333361]
 [ 2.1697327 ]
 [ 0.36234886]]
Loss:      805.2425921366158
Alpha:     6.756339739298577e-06
Iteration: 4049000
Coef:      [[-0.30355807]
 [ 2.16995573]
 [ 0.36234863]]
Loss:      805.24252710482
Alpha:     5.580356943653574e-06
Iteration: 4050000
Coef:      [[-0.30378384]
 [ 2.17017921]
 [ 0.3623439 ]]
Loss:      805.2424659240023
Alpha:     6.65753290263053e-06
Iteration: 4051000
Coef:      [[-0.30400835]
 [ 2.17040236]
 [ 0.36234401]]
Loss:      805.2424020342276
Alpha:     5.498748049140187e-06
Iteration: 4052000
Coef:      [[-0.3042

Iteration: 4105000
Coef:      [[-0.31615012]
 [ 2.18247132]
 [ 0.362355  ]]
Loss:      805.2390203051244
Alpha:     5.335982426319094e-06
Iteration: 4106000
Coef:      [[-0.31637486]
 [ 2.18269565]
 [ 0.36236013]]
Loss:      805.2389589934346
Alpha:     6.365986787185535e-06
Iteration: 4107000
Coef:      [[-0.31660053]
 [ 2.18291901]
 [ 0.36235523]]
Loss:      805.2388950204461
Alpha:     7.594812826731243e-06
Iteration: 4108000
Coef:      [[-0.31682477]
 [ 2.18314201]
 [ 0.36235601]]
Loss:      805.23883269201
Alpha:     6.272888594823525e-06
Iteration: 4109000
Coef:      [[-0.31704965]
 [ 2.18336647]
 [ 0.36236114]]
Loss:      805.2387702267166
Alpha:     7.483743896000872e-06
Iteration: 4110000
Coef:      [[-0.317274  ]
 [ 2.18358935]
 [ 0.36236065]]
Loss:      805.2387079079729
Alpha:     6.181151899698462e-06
Iteration: 4111000
Coef:      [[-0.31749972]
 [ 2.18381277]
 [ 0.36235587]]
Loss:      805.238644817728
Alpha:     7.374299272235662e-06
Iteration: 4112000
Coef:      [[-0.31

Iteration: 4165000
Coef:      [[-0.32963651]
 [ 2.19587678]
 [ 0.36236682]]
Loss:      805.235266119475
Alpha:     7.156016418904838e-06
Iteration: 4166000
Coef:      [[-0.32986071]
 [ 2.19609971]
 [ 0.36236741]]
Loss:      805.2352037968798
Alpha:     5.9104674206747816e-06
Iteration: 4167000
Coef:      [[-0.33008542]
 [ 2.19632396]
 [ 0.36237228]]
Loss:      805.2351410815764
Alpha:     7.051364584808369e-06
Iteration: 4168000
Coef:      [[-0.33030974]
 [ 2.19654695]
 [ 0.36237259]]
Loss:      805.2350794390402
Alpha:     5.8240308867525025e-06
Iteration: 4169000
Coef:      [[-0.33053516]
 [ 2.19677013]
 [ 0.3623681 ]]
Loss:      805.235015601585
Alpha:     6.948243212038231e-06
Iteration: 4170000
Coef:      [[-0.33075962]
 [ 2.19699314]
 [ 0.36236772]]
Loss:      805.2349541731628
Alpha:     5.73885842787959e-06
Iteration: 4171000
Coef:      [[-0.3309842 ]
 [ 2.19721731]
 [ 0.3623729 ]]
Loss:      805.2348907885128
Alpha:     6.846629918646792e-06
Iteration: 4172000
Coef:      [[-0.

Iteration: 4225000
Coef:      [[-0.34311603]
 [ 2.20927637]
 [ 0.36238371]]
Loss:      805.2315166529227
Alpha:     6.643966335413913e-06
Iteration: 4226000
Coef:      [[-0.34334027]
 [ 2.20949933]
 [ 0.36238427]]
Loss:      805.2314533761571
Alpha:     5.4875428270095876e-06
Iteration: 4227000
Coef:      [[-0.34356575]
 [ 2.20972244]
 [ 0.36237909]]
Loss:      805.2313918683734
Alpha:     6.546802882736626e-06
Iteration: 4228000
Coef:      [[-0.34379005]
 [ 2.20994532]
 [ 0.36237887]]
Loss:      805.2313285820402
Alpha:     5.407291275320542e-06
Iteration: 4229000
Coef:      [[-0.34401446]
 [ 2.21016946]
 [ 0.36238475]]
Loss:      805.2312669895703
Alpha:     6.451060378941316e-06
Iteration: 4230000
Coef:      [[-0.34423866]
 [ 2.21039228]
 [ 0.36238479]]
Loss:      805.2312029232769
Alpha:     5.328213347555984e-06
Iteration: 4231000
Coef:      [[-0.34446418]
 [ 2.21061545]
 [ 0.36237976]]
Loss:      805.2311419374589
Alpha:     6.3567180436247215e-06
Iteration: 4232000
Coef:      [[

In [12]:
sum((clf.predict(X) - y)**2)

805.2283536684514

In [15]:
clf.coef

matrix([[-0.35448323],
        [ 2.22057545],
        [ 0.3623942 ]])